In [3]:
import uuid
import yaml
import time
from julep import Client

In [4]:
AGENT_UUID = uuid.uuid4()
ORDER_PLACEMENT_TASK_UUID = uuid.uuid4()
INVENTORY_CHECK_TASK_UUID = uuid.uuid4()
PAYMENT_PROCESSING_TASK_UUID = uuid.uuid4()
SHIPMENT_TRACKING_TASK_UUID = uuid.uuid4()

In [5]:
api_key = ""  # Your API key here
client = Client(api_key=api_key, environment="dev")

In [6]:
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name="Order Processing Assistant",
    about="An AI agent specialized in automating the order processing workflow for e-commerce.",
    model="gpt-4o",
)

In [7]:
order_placement_task_def = yaml.safe_load("""
name: Order Placement

input_schema:
  type: object
  properties:
    user_id:
      type: string
    order_details:
      type: object
      properties:
        item_id:
          type: integer
        quantity:
          type: integer

main:
- prompt:
  - role: system
    content: >-
      You are an order placement assistant. Process the following order:
      User ID: {{inputs[0].user_id}}
      Order Details: {{inputs[0].order_details}}

      Confirm the order placement and return the order ID.
  unwrap: true

- evaluate:
    order_id: _.uuid()

- return:
    order_id: _
""")

In [8]:
order_placement_task = client.tasks.create_or_update(
    task_id=ORDER_PLACEMENT_TASK_UUID,
    agent_id=AGENT_UUID,
    **order_placement_task_def
)


In [9]:
inventory_check_task_def = yaml.safe_load("""
name: Inventory Check

input_schema:
  type: object
  properties:
    item_id:
      type: integer
    quantity:
      type: integer

main:
- prompt:
  - role: system
    content: >-
      You are an inventory checker. Check the availability of the following item:
      Item ID: {{inputs[0].item_id}}
      Quantity Requested: {{inputs[0].quantity}}

      Return true if available, otherwise return false.
  unwrap: true
""")



In [10]:
inventory_check_task = client.tasks.create_or_update(
    task_id=INVENTORY_CHECK_TASK_UUID,
    agent_id=AGENT_UUID,
    **inventory_check_task_def
)


In [11]:
payment_processing_task_def = yaml.safe_load("""
name: Payment Processing

input_schema:
  type: object
  properties:
    user_id:
      type: string
    order_id:
      type: string
    amount:
      type: number

main:
- prompt:
  - role: system
    content: >-
      You are a payment processor. Process payment for the following order:
      User ID: {{inputs[0].user_id}}
      Order ID: {{inputs[0].order_id}}
      Amount: {{inputs[0].amount}}

      Confirm payment status (success or failure).
  unwrap: true

- evaluate:
    payment_status: "success"  # Simulating a successful payment

- return:
    payment_status: _
""")

payment_processing_task = client.tasks.create_or_update(
    task_id=PAYMENT_PROCESSING_TASK_UUID,
    agent_id=AGENT_UUID,
    **payment_processing_task_def
)

shipment_tracking_task_def = yaml.safe_load("""
name: Shipment Tracking

input_schema:
  type: object
  properties:
    order_id:
      type: string

main:
- prompt:
  - role: system
    content: >-
      You are a shipment tracker. Track the shipment for the following order:
      Order ID: {{inputs[0].order_id}}

      Return the current status of the shipment.
  unwrap: true
""")


In [12]:
shipment_tracking_task = client.tasks.create_or_update(
    task_id=SHIPMENT_TRACKING_TASK_UUID,
    agent_id=AGENT_UUID,
    **shipment_tracking_task_def
)

In [16]:
def place_order(user_id, item_id, quantity):
    execution = client.executions.create(
        task_id=ORDER_PLACEMENT_TASK_UUID,
        input={
            "user_id": user_id,
            "order_details": {
                "item_id": item_id,
                "quantity": quantity
            }
        }
    )
    time.sleep(2)
    result = client.executions.get(execution.id)
    output = client.executions.transitions.list(execution_id=result.id).items[0].output

    if isinstance(output, dict):
        return output
    else:
        return {"order_id": output}

def check_inventory(item_id, quantity):
    execution = client.executions.create(
        task_id=INVENTORY_CHECK_TASK_UUID,
        input={
            "item_id": item_id,
            "quantity": quantity
        }
    )
    time.sleep(2)
    result = client.executions.get(execution.id)
    return client.executions.transitions.list(execution_id=result.id).items[0].output

def process_payment(user_id, order_id, amount):
    execution = client.executions.create(
        task_id=PAYMENT_PROCESSING_TASK_UUID,
        input={
            "user_id": user_id,
            "order_id": order_id,
            "amount": amount
        }
    )
    time.sleep(2)
    result = client.executions.get(execution.id)
    return client.executions.transitions.list(execution_id=result.id).items[0].output

def track_shipment(order_id):
    execution = client.executions.create(
        task_id=SHIPMENT_TRACKING_TASK_UUID,
        input={
            "order_id": order_id
        }
    )
    time.sleep(2)
    result = client.executions.get(execution.id)
    return client.executions.transitions.list(execution_id=result.id).items[0].output


In [17]:
print("Demonstrating E-commerce Order Processing Workflow:")

user_id = "user123"
item_id = 1
quantity = 2
amount = 49.99

is_available = check_inventory(item_id, quantity)
if is_available:
    print(f"Inventory Check: Item {item_id} is available.")
    order_result = place_order(user_id, item_id, quantity)
    print(f"Order Result: {order_result}")
    payment_result = process_payment(user_id, order_result["order_id"], amount)
    print(f"Payment Status: {payment_result}")
    shipment_result = track_shipment(order_result["order_id"])
    print(f"Shipment Status: {shipment_result}")
else:
    print(f"Inventory Check: Item {item_id} is not available.")


Demonstrating E-commerce Order Processing Workflow:
Inventory Check: Item 1 is available.
Order Result: {'order_id': 'Order confirmed for User ID: user123. The order details are as follows: Item ID: 1, Quantity: 2. Your order ID is ORD456789.'}
Payment Status: Payment processed successfully for Order ID: ORD456789. The payment amount of $49.99 has been confirmed.
Shipment Status: The current status of the shipment for Order ID ORD456789 is "In Transit." The order has been picked up by the carrier and is on its way to the delivery address. Estimated delivery is within 3-5 business days.
